<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-0723457ef703539d'
PROJECT = 'qwiklabs-gcp-0723457ef703539d'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

Creating gs://qwiklabs-gcp-0723457ef703539d/...


<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [5]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [6]:
df = bigquery.Client().query(query + " LIMIT 100").to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.131954,True,32,1,28.0,6691862025345277042
1,7.374463,True,33,1,39.0,7146494315947640619
2,6.750554,False,20,1,NaN,2126480030009879160
3,8.062305,False,26,1,40.0,7146494315947640619
4,7.484694,False,24,1,39.0,3408502330831153141


In [15]:
train_query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) < 8 AND RAND() < 0.0005
"""

test_query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 AND MOD(ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))), 10) >= 8 AND RAND() < 0.0005
"""

In [16]:
train_df = bigquery.Client().query(train_query).to_dataframe()
test_df = bigquery.Client().query(test_query).to_dataframe()

In [17]:
len(train_df), len(test_df)

(13357, 3030)

In [18]:
train_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.876218,False,22,1,40.0,774501970389208065
1,6.768191,True,40,1,38.0,774501970389208065
2,8.840537,True,40,1,38.0,774501970389208065
3,6.311835,False,28,1,41.0,774501970389208065
4,7.874912,False,22,1,38.0,774501970389208065


In [19]:
test_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.473671,True,23,1,39.0,3545707052733304728
1,6.999677,False,26,1,38.0,1451354159195218418
2,6.000983,False,39,1,35.0,7361509899869211058
3,4.312242,False,27,1,36.0,3572456083892950068
4,5.923821,True,26,1,39.0,7586701093880244808


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

## Filter Data

In [31]:
def cleanup_dataframe(df):
    df = df[df.weight_pounds > 0][df.mother_age > 0][df.gestation_weeks > 0]
    twins = dict(zip([1,2,3,4,5],  ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
    df['plurality'].replace(twins, inplace=True)
    return df

def simulate_no_ultrasounds(df):
    no_ultrasounds = df.copy(deep=True)
    no_ultrasounds['is_male']='Unknown'
    no_ultrasounds.head()
    return no_ultrasounds

In [32]:
traind_df = cleanup_dataframe(train_df)
test_df = cleanup_dataframe(test_df)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [33]:
traind_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.876218,False,22,Single(1),40.0,774501970389208065
1,6.768191,True,40,Single(1),38.0,774501970389208065
2,8.840537,True,40,Single(1),38.0,774501970389208065
3,6.311835,False,28,Single(1),41.0,774501970389208065
4,7.874912,False,22,Single(1),38.0,774501970389208065


In [34]:
test_df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.473671,True,23,Single(1),39.0,3545707052733304728
1,6.999677,False,26,Single(1),38.0,1451354159195218418
2,6.000983,False,39,Single(1),35.0,7361509899869211058
3,4.312242,False,27,Single(1),36.0,3572456083892950068
4,5.923821,True,26,Single(1),39.0,7586701093880244808


In [36]:
train_no_us = simulate_no_ultrasounds(traind_df)
test_no_us = simulate_no_ultrasounds(test_df)

In [37]:
train_no_us.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.876218,Unknown,22,Single(1),40.0,774501970389208065
1,6.768191,Unknown,40,Single(1),38.0,774501970389208065
2,8.840537,Unknown,40,Single(1),38.0,774501970389208065
3,6.311835,Unknown,28,Single(1),41.0,774501970389208065
4,7.874912,Unknown,22,Single(1),38.0,774501970389208065


In [38]:
test_no_us.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.473671,Unknown,23,Single(1),39.0,3545707052733304728
1,6.999677,Unknown,26,Single(1),38.0,1451354159195218418
2,6.000983,Unknown,39,Single(1),35.0,7361509899869211058
3,4.312242,Unknown,27,Single(1),36.0,3572456083892950068
4,5.923821,Unknown,26,Single(1),39.0,7586701093880244808


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [42]:
traind_df.to_csv('train.csv', index=False, header=False)
test_df.to_csv('eval.csv', index=False, header=False)

In [43]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

  2998 eval.csv
 13248 train.csv
 16246 total
==> eval.csv <==
7.4736706818,True,23,Single(1),39.0,3545707052733304728
6.9996768185,False,26,Single(1),38.0,1451354159195218418
6.0009827716399995,False,39,Single(1),35.0,7361509899869211058
4.31224184472,False,27,Single(1),36.0,3572456083892950068
5.9238209799399995,True,26,Single(1),39.0,7586701093880244808
7.31273323054,True,19,Single(1),38.0,6544754656115871769
9.93843877096,False,39,Single(1),39.0,1451354159195218418
7.9586876582,False,36,Single(1),39.0,1443901198490054949
9.18666245754,False,35,Single(1),35.0,2995620979373137889
6.87401332916,True,21,Single(1),38.0,2768780346491825728

==> train.csv <==
6.87621795178,False,22,Single(1),40.0,774501970389208065
6.7681914434,True,40,Single(1),38.0,774501970389208065
8.8405367062,True,40,Single(1),38.0,774501970389208065
6.3118345610599995,False,28,Single(1),41.0,774501970389208065
7.87491199864,False,22,Single(1),38.0,774501970389208065
9.12493302418,True,34,Single(1),39.0,774501970389

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License